In [15]:
import pandas as pd

scientist_data = pd.read_csv('/Users/wcl/Downloads/assignment_cvand/scientist.csv')
inst_wos_dict = pd.read_csv('/Users/wcl/Downloads/assignment_cvand/inst_wos_dict.csv')
merged_data = pd.merge(scientist_data, inst_wos_dict, on='inst', how='inner')
cddt_paper = pd.read_csv('/Users/wcl/Downloads/assignment_cvand/cddt_paper.csv')
print(cddt_paper.head())
cite_paper = pd.read_csv('/Users/wcl/Downloads/assignment_cvand/cite.csv')
print(cite_paper.head())


def filter_matched_papers(inst, startyear,endyear, type_1, cddt_paper):
    filtered_df = cddt_paper[cddt_paper['addr'].str.contains(inst, case=False)]
    matched_papers = filtered_df[(filtered_df['pub_year'].between(startyear, endyear)) & (filtered_df['type_1'] == 1)]
    return matched_papers

def second_filter_related_papers(matched_papers, cite_file):
    unique_ids = list(set(matched_papers['ut_char'].tolist()))
    related_papers = cite_paper[(cite_paper['citing_ut'].isin(unique_ids)) | (cite_paper['cited_ut'].isin(unique_ids))]
    paper_ids = list(set(related_papers['citing_ut'].tolist() + related_papers['cited_ut'].tolist()))
    related_paper_df = pd.DataFrame({'ut_char': paper_ids})
    return related_paper_df 

/var/folders/cy/rgx587b13vg099yjcvkjcbjr0000gn/T/ipykernel_50581/2304053075.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  cddt_paper = pd.read_csv('/Users/wcl/Downloads/assignment_cvand/cddt_paper.csv')


        ut_char uniqueID                                               addr  \
0  207610701393    0_394  Natl Cheng Kung Univ, Med Ctr, Dept Med, Taina...   
1  207610701495    0_117                     Merck Res Labs, Rahway, NJ USA   
2  207783700157    1_449  Zhejiang Prov Peoples Hosp, Dept Gastroenterol...   
3  207783700182    0_394  Second Mil Med Univ, Dept Gastroenterol, Diges...   
4  207910400275    1_449  Toyohashi Univ Technol, Dept Informat Comp Sci...   

            item_type  pub_year  type  
0    Meeting Abstract    2008.0     2  
1    Meeting Abstract    2008.0     2  
2  Editorial Material    2009.0     2  
3  Editorial Material    2009.0     2  
4    Meeting Abstract    1998.0     2  


/var/folders/cy/rgx587b13vg099yjcvkjcbjr0000gn/T/ipykernel_50581/2304053075.py:8: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  cite_paper = pd.read_csv('/Users/wcl/Downloads/assignment_cvand/cite.csv')


     citing_ut         cited_ut
0  60356100016  A1993KT46100012
1  70587000013  A1992HU46000003
2  70940200019  A1996VF80400056
3  70948600013  A1995RR73900017
4  70948600023  A1993MM89300019


In [12]:
paper_info = []
for i,v in merged_data.iterrows():
    ut_char = v['ut_char']
    uniqueID = v['uniqueID']
    addr = v['addr']
    type_1 = v['type']
    pub_year = v['pub_year']
    first_step_match = filter_matched_papers(inst, startyear,endyear, type_1, cddt_paper)
    print(first_step_match)
    second_step_match = second_filter_related_papers(first_step_match, cite_paper)
    merged_data_list = pd.merge(second_step_match, cddt_paper, on='ut_char', how='inner')['uniqueID'].tolist
    merged_data_list_df = pd.DataFrame({'uniqueID': merged_data_list})
    paper_info_final = filter_matched_papers(inst, startyear,endyear, type_1, merged_data_list_df)['uniqueID'].tolist()
    for item_ut_char in paper_info_final:
        paper_info.append([uniqueID,item_ut_char])